<a href="https://colab.research.google.com/github/viniciusgcjr/curso-python/blob/main/Curso_Apac_INTRO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Slack

In [ ]:
import requests
import json

[Tutorial - Slack API com Python](https://practicaldatascience.co.uk/data-science/how-to-send-a-slack-message-in-python-using-webhooks)

In [ ]:
def send_slack_message(payload, webhook):
    """Send a Slack message to a channel via a webhook. 
    
    Args:
        payload (dict): Dictionary containing Slack message, i.e. {"text": "This is a test"}
        webhook (str): Full Slack webhook URL for your chosen channel. 
    
    Returns:
        HTTP response code, i.e. <Response [503]>
    """

    return requests.post(webhook, json.dumps(payload))

In [ ]:
webhook = "<TOKEN SLACK>"
mensagem = 'Seja bem-vindo!'
payload = {"text": mensagem}
send_slack_message(payload, webhook)

## Importando dados do google planilhas

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

worksheet = gc.open('Formulario_inicial_respostas').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
#print(rows)

# Convert to a DataFrame and render.
import pandas as pd
respostas = pd.DataFrame.from_records(rows)
respostas.columns = respostas.iloc[0]
respostas.drop(respostas.index[0], inplace = True)
respostas

In [ ]:
respostas['Já estudou programação?'].value_counts().plot(kind='bar', title='Quem Estudou Programação')

In [ ]:
respostas['Qual o email que foi cadastrado no slack?'].values

## Salvar dados no google planilhas

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

sh = gc.create('A new spreadsheet')

# Open our new sheet and add some data.
worksheet = gc.open('A new spreadsheet').sheet1

cell_list = worksheet.range('A1:C2')

import random
for cell in cell_list:
  cell.value = random.randint(1, 10)

worksheet.update_cells(cell_list)
# Go to https://sheets.google.com to see your new spreadsheet.

# Formulário Inicial

# Baixar pdfs do site da Apac

In [ ]:
import requests


def download_pdf(url, file_name):

    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        with open(file_name, "wb") as f:
            f.write(response.content)
    else:
        print(response.status_code)

In [ ]:
download_pdf('https://www.apac.pe.gov.br/uploads/Boletim-Pluviometrico-29-09-2022.pdf', 'boletim_chuva_29.pdf')

In [ ]:
url = 'https://www.apac.pe.gov.br/uploads/Boletim-Pluviometrico-30-09-2022.pdf'
arquivo = 'boletim_chuva_30.pdf'
download_pdf(url, arquivo)


# Baixar dados do INMET

In [ ]:
import json
import pandas as pd

In [ ]:
def recuperarDados(estadoEstacao):
  response = requests.get(f'https://apitempo.inmet.gov.br/estacao/diaria/2020-01-01/2020-12-30/{estadoEstacao}')
  data = json.loads(response.text)
  df = pd.json_normalize(data)
  return df

In [ ]:
recife = recuperarDados("A301")
recife

# FTP

[Referência](https://docs.python.org/pt-br/3.7/library/ftplib.html#module-ftplib)

In [ ]:
import pandas as pd
import ftplib
import os
ftp = ftplib.FTP('<endereço ftp>', '<usuario>', '<senha>') 
dir = 'APAC'
ftp.cwd(dir)
for i in range(1, 6):
  estacao = dir + str(i).zfill(2)
  ftp.cwd(estacao)

  arquivo = estacao + '.csv'

  pathdst = './'

  with open(pathdst + os.sep + arquivo, 'wb') as file:
    ftp.retrbinary("RETR Diaria.dat", file.write)
  
  ftp.cwd('../')

In [5]:
import glob
filesglob = glob.glob('APAC*.csv')
filesglob

['APAC04.csv', 'APAC05.csv', 'APAC03.csv', 'APAC01.csv', 'APAC02.csv']

In [6]:
df_header = pd.read_csv('APAC01.csv', skiprows=1, header=0, encoding = "ISO-8859-1", delimiter=',').head()
df_header.head()

,TIMESTAMP,RECORD,Enc_RH_Min,Enc_RH_TMn,BattV_Min,ChgInV_Min,ChgInV_TMn,ChgInI_Min,ChgInI_TMn,BP_mbar,...,Vel_Vento_S_WVT_10,Dir_Vento_D1_WVT_10,Dir_Vento_SD1_WVT_10,Wind_Direction_10,Wind_Speed_10_Max,Wind_Speed_10_TMx,Wind_Speed_10_Min,Wind_Speed_10_TMn,ETos,Rso
0,TS,RN,%,NaN,Volts,V,NaN,A,NaN,mbar,...,m/s,Deg,Deg,Deg,m/s,NaN,m/s,NaN,mm,MJ/m²
1,NaN,NaN,Min,TMn,Min,Min,TMn,Min,TMn,Smp,...,WVc,WVc,WVc,Smp,Max,TMx,Min,TMn,ETXs,Rso
2,2019-07-25 00:00:00,110,63.66,2019-07-24 15:47:00,12.62,0.047,2019-07-24 02:46:30,0,2019-07-24 00:00:30,616.9,...,3.702,146,19.88,178,9.2,2019-07-24 08:02:00,0.5,2019-07-24 15:27:30,8.16,3.331
3,2019-07-26 00:00:00,111,55.66,2019-07-25 14:25:00,12.57,0.048,2019-07-25 02:52:00,0,2019-07-25 00:00:30,500,...,3.662,137.3,22.14,124,10.5,2019-07-25 12:44:30,0.4,2019-07-25 04:03:30,19.68,3.34
4,2019-07-27 00:00:00,112,63.13,2019-07-26 15:15:00,12.64,0.047,2019-07-26 05:51:00,0,2019-07-26 00:00:30,722.6,...,3.622,135.7,22.94,141,9.2,2019-07-26 14:12:00,0.5,2019-07-26 07:24:30,13.25,3.343


# SQL

[Referência](https://docs.python.org/pt-br/3.7/library/sqlite3.html#module-sqlite3)

In [ ]:
import sqlite3
conn = sqlite3.connect('example.db')



In [ ]:
c = conn.cursor()

# Create table
c.execute('''CREATE TABLE stocks
             (date text, trans text, symbol text, qty real, price real)''')

# Insert a row of data
c.execute("INSERT INTO stocks VALUES ('2020-01-05','BUY','RHAT',100,35.14)")

# Save (commit) the changes
conn.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.


In [ ]:
conn = sqlite3.connect('example.db')
c = conn.cursor()
c.execute("SELECT * FROM stocks")
print(c.fetchall())
conn.commit()
conn.close()

# PDF e WORD

### WORD

In [ ]:
pip install python-docx

In [ ]:
import docx
doc = docx.Document()
doc.add_paragraph('Hello world!')
doc.save('helloworld.docx')



In [ ]:
download_pdf('https://www.apac.pe.gov.br/images/Marca_Apac_2020_hor_positiva.png', 'logo_apac')

In [ ]:
from docx import Document
from docx.shared import Inches

document = Document()

document.add_heading('Document Title', 0)

p = document.add_paragraph('A plain paragraph having some ')
p.add_run('bold').bold = True
p.add_run(' and some ')
p.add_run('italic.').italic = True

document.add_heading('Heading, level 1', level=1)
document.add_paragraph('Intense quote', style='Intense Quote')

document.add_paragraph(
    'first item in unordered list', style='List Bullet'
)
document.add_paragraph(
    'first item in ordered list', style='List Number'
)

document.add_picture('/content/logo_apac', width=Inches(1.25))

records = (
    (3, '101', 'Spam'),
    (7, '422', 'Eggs'),
    (4, '631', 'Spam, spam, eggs, and spam')
)

table = document.add_table(rows=1, cols=3)
hdr_cells = table.rows[0].cells
hdr_cells[0].text = 'Qty'
hdr_cells[1].text = 'Id'
hdr_cells[2].text = 'Desc'
for qty, id, desc in records:
    row_cells = table.add_row().cells
    row_cells[0].text = str(qty)
    row_cells[1].text = id
    row_cells[2].text = desc

document.add_page_break()

document.save('demo.docx')

https://python-docx.readthedocs.io/en/latest/

### PDF

### Extrair Informações

In [ ]:
pip install PyPDF2

In [ ]:
import PyPDF2

In [ ]:
pdfFileObj = open('/content/2022.0126 Reservatório TJ JULHO.pdf', 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
pdfReader.numPages

In [ ]:
pageObj = pdfReader.getPage(0)
text_from_pdf = pageObj.extractText()
text_from_pdf

### Regular Expressions

[Referência](https://docs.python.org/3/library/re.html)

In [ ]:
import re

codigo = re.findall('(Código Ponto de Coleta) (?P<Code>\w{2}-\d{2})', text_from_pdf)
print(codigo[0][1])
data_coleta = re.findall('(Data da Coleta) (?P<Data>\d{2}\/\d{2}\/\d{4})', text_from_pdf)
print(data_coleta[0][1]) 

In [ ]:
import shutil
shutil.move('/content/2022.0126 Reservatório TJ JULHO.pdf', f'/content/Reservatorio_{codigo[0][1]}.pdf')

## Extrair Tabelas

In [ ]:
pip install -q tabula-py

In [ ]:
import tabula

#tabula.environment_info()

In [ ]:

pdf_path = "/content/CB 54- 692-2022- RE 194.658 AGENCIA PERNAMBUCANA DE AGUAS E CLIMA APAC (1).pdf"

dfs = tabula.read_pdf(pdf_path)
print(len(dfs))
dfs[0].to_csv('teste')

### Combinar PDFs

In [ ]:
import PyPDF2
pdf1File = open('boletim_chuva_29.pdf', 'rb')
pdf2File = open('boletim_chuva_30.pdf', 'rb') 
pdf1Reader = PyPDF2.PdfFileReader(pdf1File) 
pdf2Reader = PyPDF2.PdfFileReader(pdf2File) 
pdfWriter = PyPDF2.PdfFileWriter()
for pageNum in range(pdf1Reader.numPages - 3 ): #slice das pgs desejadas
  pageObj = pdf1Reader.getPage(pageNum)
  pdfWriter.addPage(pageObj)
for pageNum in range(pdf2Reader.numPages - 2): #slice das pgs desejadas
  pageObj = pdf2Reader.getPage(pageNum)
  pdfWriter.addPage(pageObj) 
pdfOutputFile = open('boletinscombinados.pdf', 'wb')
pdfWriter.write(pdfOutputFile)
pdfOutputFile.close()
pdf1File.close()
pdf2File.close()

# Satélite

In [ ]:
!wget -c https://noaa-goes16.s3.amazonaws.com/ABI-L2-CMIPF/2019/198/12/OR_ABI-L2-CMIPF-M6C13_G16_s20191981200396_e20191981210116_c20191981210189.nc 

In [ ]:

from netCDF4 import Dataset     #Para ler arquivo de satélite
import matplotlib.pyplot as plt  
from datetime import datetime   

file = Dataset("/content/OR_ABI-L2-CMIPF-M6C13_G16_s20191981200396_e20191981210116_c20191981210189.nc")
 
 
data = file.variables['CMI'][:] - 273.15 #ler tabela de dados do satélite

#Plotar mapa
plt.figure(figsize=(7,7))
plt.imshow(data, vmin=-80, vmax=40, cmap='jet')
plt.colorbar(label='Brightness Temperature (°C)', extend='both', orientation='horizontal', pad=0.05, fraction=0.05)

date = (datetime.strptime(file.time_coverage_start, '%Y-%m-%dT%H:%M:%S.%fZ'))
	
plt.title('GOES-16 Band 13 ' + date.strftime('%Y-%m-%d %H:%M') + ' UTC', fontweight='bold', fontsize=10, loc='left')
plt.title('Full Disk', fontsize=10, loc='right')

plt.savefig('/content/satelite_goes.png')
 
plt.show()

# Mapas

In [ ]:
pontos = {'lat': [-8.044355,-8.045071, -8.045351, -8.044688],	'lon': [-34.875557, -34.876049, -34.875596, -34.875131]}
apac = pd.DataFrame(data=pontos)
apac


In [ ]:
import folium
map = folium.Map(location = apac.iloc[0], zoom_start = 20)
for _, local in apac.iterrows():
  folium.Marker(location=[local['lat'], local['lon']],
                popup =      
                            "Latitude: " + str(local['lat']) + '<br>' +
                            "Longitude: " + str(local['lon']),
                tooltip='Detalhes').add_to(map)

map